# Train the embedding model

## Processing Setup

In [ ]:
# Google colab
import os
from google.colab import drive
drive.mount('/content/drive')
project_home = '/content/drive/MyDrive/Projects/verge'
os.chdir(project_home)

In [ ]:
# Local processing setup
# project_home = '..'

## Notebook Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from typing import List, Tuple, Optional

import pickle
import pandas as pd

import sys
sys.path.append('%s/03-embeddings' % project_home)
from embedderv5 import *

## Parameters

In [ ]:
# The name of the ROI to use.
roi_name = 'newengland'

# The name of the general-purpose data directory.
data_home = '%s/data' % (project_home)

# The name of the ROI-specific data directory.
roi_home = '%s/data/%s' % (project_home, roi_name)

# The unique identifier of the model to be used.
run_id = '201b'

# Identifier of the splits file.
splits_id = '201'

# What type of device to train on.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device', device)

# Number of epochs for whch to train the model.
num_epochs = 10


## Load and organize data
We have two data sources that we need to associate with one another:
a set of initial embeddings ("initials") and a set of feature vectors
to be used for similarity assessments ("features").

In [ ]:
# We will divide into training and validation sets based on AOI.
# The splits have already been determined, before training the initial MGM.
# Here we look them up and re-organize things a bit.
fname = '%s/models/splits-%s.csv' % (roi_home, splits_id)
splits = pd.read_csv(fname)
print('%d splits' % len(splits))
splits.head(3)

splits_lookup = {
    '%s : %s' % (z['aoi_tag'], z['tile_tag']): z['split']
    for k, z in splits.iterrows()
}
print('%d elements in splits lookup' % len(splits_lookup))


In [ ]:
# Get a list of tiles.
fname = '%s/tiles.csv' % roi_home
tile_info = pd.read_csv(fname)
print('%d tiles' % len(tile_info))
tile_info.head(3)

In [ ]:
# Get the list of AOI tags.
aoi_tags = np.unique(tile_info['aoi_tag'])
print('%d unique AOIs' % len(aoi_tags))

In [ ]:
# Load initial embeddings. Put them into a lookup table based on aoi/tile identifiers.
embeddings_lookup = {}

for k, aoi_tag in enumerate(aoi_tags):

    if k % 20 == 0:
      print('%d/%d' % (k, len(aoi_tags)))

    fname = '%s/initials/%s.pkl' % (roi_home, aoi_tag)
    with open(fname, 'rb') as source:
        a = pickle.load(source)
    for b in a:
        key = '%s : %s' % (b['aoi_tag'], b['tile_tag'])
        e = b['embedding']
        embeddings_lookup[key] = e

print('%d total embeddings' % len(embeddings_lookup))


In [ ]:
embedding_dim = e.shape[-1]
print('dimension of embeddings is %d' % embedding_dim)

In [ ]:
# Load initial Features. Ditto.
features_lookup = {}

for k, aoi_tag in enumerate(aoi_tags):

    if k % 20 == 0:
      print('%d/%d' % (k, len(aoi_tags)))

    fname = '%s/features/%s.pkl' % (roi_home, aoi_tag)
    with open(fname, 'rb') as source:
        a = pickle.load(source)
    for b in a:
        key = '%s : %s' % (b['aoi_tag'], b['tile_tag'])
        f = b['features']
        features_lookup[key] = f

print('%d total feature vectors' % len(features_lookup))


In [ ]:
# Organize the data the way the model expects it.
train_sequences = []
train_features = []
val_sequences = []
val_features = []
for key in features_lookup.keys():
    f = features_lookup[key]
    if key in embeddings_lookup:
        e = embeddings_lookup[key].squeeze().detach().cpu().numpy()
        if splits_lookup[key] == 'train':
            train_sequences.append(e)
            train_features.append(f)
        elif splits_lookup[key] == 'val':
            val_sequences.append(e)
            val_features.append(f)
    else:
        print('key mismatch: %s' % key)

print('%d training sequences' % len(train_sequences))
print('%d val sequences' % len(val_sequences))

#


In [ ]:
# # Organize the data the way the model expects it.
# sequences = []
# similarity_features = []
# for key in features_lookup.keys():
#     f = features_lookup[key]
#     if key in embeddings_lookup:
#         e = embeddings_lookup[key].squeeze().detach().cpu().numpy()
#         sequences.append(e)
#         similarity_features.append(f)
#     else:
#         print('key mismatch: %s' % key)

# print(len(sequences))
# print(type(sequences))
# print(type(sequences[0]))
# print(sequences[0].shape)


In [ ]:
# # # This was the Claude-generated code to generate test data for the model.
# sequences, similarity_features = generate_sample_data(
#     num_instances=1000, min_R=5, max_R=20, C=32, similarity_dim=16
# )
# print(type(sequences))
# print(type(sequences[0]))
# print(sequences[0].shape)

## Model
The model code, including data loaders, the model itself, loss function, and all that,
were generated by Claude via a lot of iterative prompting and debugging.


In [ ]:
# Create dataset and dataloader with explicit triplet sampling
train_dataset = ContrastivePairDataset(
    train_sequences,
    train_features,
    similarity_threshold=0.5,  # Adjust based on your similarity features
    num_negatives=2  # Number of negatives per anchor
)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=triplet_collate_fn)

val_dataset = ContrastivePairDataset(
    val_sequences,
    val_features,
    similarity_threshold=0.5,  # Adjust based on your similarity features
    num_negatives=2  # Number of negatives per anchor
)
val_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=triplet_collate_fn)



In [ ]:
# Initialize model
model = PermutationInvariantModel(
    input_dim=embedding_dim,
    hidden_dim=128,
    embedding_dim=embedding_dim,
    num_attention_heads=8,
    num_linear_layers=3,
    dropout=0.1
)

print(f"Model has {sum(p.numel() for p in model.parameters())} parameters")


In [ ]:
def train_model_with_triplets(
    model, train_loader, val_loader,
    num_epochs=100, learning_rate=1e-3,
    device='cpu'
):
    """
    Training loop for the permutation-invariant model with explicit triplet sampling.
    """

    train_loss_history = []
    val_loss_history = []
    min_avg_val_loss = None

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    criterion = TripletContrastiveLoss(margin=0.5, temperature=0.1)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

    model.train()

    for epoch in range(num_epochs):
        total_loss = 0.0
        num_batches = 0

        for batch_idx, batch_data in enumerate(train_loader):
            # Unpack batch data
            anchor_seqs, anchor_masks, anchor_sims = batch_data['anchor']
            pos_seqs, pos_masks, pos_sims = batch_data['positive']
            neg_seqs, neg_masks, neg_sims, neg_batch_indices = batch_data['negatives']

            # Move to device
            anchor_seqs, anchor_masks = anchor_seqs.to(device), anchor_masks.to(device)
            pos_seqs, pos_masks = pos_seqs.to(device), pos_masks.to(device)
            neg_seqs, neg_masks = neg_seqs.to(device), neg_masks.to(device)
            neg_batch_indices = neg_batch_indices.to(device)

            optimizer.zero_grad()

            # Forward pass
            anchor_emb = model(anchor_seqs, anchor_masks)
            pos_emb = model(pos_seqs, pos_masks)
            neg_emb = model(neg_seqs, neg_masks)

            # Compute loss
            loss = criterion(anchor_emb, pos_emb, neg_emb, neg_batch_indices)

            # Backward pass
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()
            num_batches += 1


        # Proces the validation dataset.
        total_val_loss = 0.0
        num_val_batches = 0

        model.eval()
        with torch.no_grad():
          for batch_idx, batch_data in enumerate(val_loader):
              # Unpack batch data
              anchor_seqs, anchor_masks, anchor_sims = batch_data['anchor']
              pos_seqs, pos_masks, pos_sims = batch_data['positive']
              neg_seqs, neg_masks, neg_sims, neg_batch_indices = batch_data['negatives']

              # Move to device
              anchor_seqs, anchor_masks = anchor_seqs.to(device), anchor_masks.to(device)
              pos_seqs, pos_masks = pos_seqs.to(device), pos_masks.to(device)
              neg_seqs, neg_masks = neg_seqs.to(device), neg_masks.to(device)
              neg_batch_indices = neg_batch_indices.to(device)

              # Forward pass
              anchor_emb = model(anchor_seqs, anchor_masks)
              pos_emb = model(pos_seqs, pos_masks)
              neg_emb = model(neg_seqs, neg_masks)

              # Compute loss
              val_loss = criterion(anchor_emb, pos_emb, neg_emb, neg_batch_indices)
              total_val_loss += val_loss.item()
              num_val_batches += 1

        scheduler.step()

        avg_loss = total_loss / num_batches
        avg_val_loss = total_val_loss / num_val_batches
        print(f'Epoch {epoch} - Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

        train_loss_history.append(avg_loss)
        val_loss_history.append(avg_val_loss)

        if min_avg_val_loss is None or avg_val_loss < min_avg_val_loss:
            min_avg_val_loss = avg_val_loss
            model_fname = '%s/models/model-%s.pth' % (roi_home, run_id)
            print('==> Saving model to %s' % model_fname)
            torch.save(model.state_dict(), model_fname)

    return train_loss_history, val_loss_history


In [ ]:
# Train the model
train_loss_history, val_loss_history = train_model_with_triplets(
    model, train_loader, val_loader, num_epochs=num_epochs, learning_rate=1e-3,
    device=device
)


In [ ]:
# Plot the trainign and validation loss history using plotly.
import plotly
from plotly.graph_objects import Scatter
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=1)

train_loss_history = np.array(train_loss_history)
val_loss_history = np.array(val_loss_history)

fig.add_trace(
    Scatter(y=train_loss_history, name='Training Loss', mode='markers+lines'),
    row=1, col=1
)

fig.add_trace(
    Scatter(y=val_loss_history, name='Validation Loss', mode='markers+lines'),
    row=1, col=1
)

fig.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    height=400,
    width=900,
)

fig.show()


In [ ]:

# Example inference
model.eval()
with torch.no_grad():
    sample_batch = next(iter(train_loader))
    # Unpack the dictionary structure from triplet data loader
    anchor_seqs, anchor_masks, anchor_sims = sample_batch['anchor']
    anchor_seqs = anchor_seqs.to(device)
    anchor_masks = anchor_masks.to(device)

    # Generate embeddings for anchor samples
    embeddings = model(anchor_seqs, anchor_masks)
    print(f"Generated embeddings shape: {embeddings.shape}")
    print(f"Sample embedding norm: {torch.norm(embeddings[0]).item():.4f}")

    # Get embeddings for positives
    pos_seqs, pos_masks, pos_sims = sample_batch['positive']
    pos_seqs, pos_masks = pos_seqs.to(device), pos_masks.to(device)
    pos_embeddings = model(pos_seqs, pos_masks)
    print(f"Positive embeddings shape: {pos_embeddings.shape}")

    # Get embeddings for negatives
    neg_seqs, neg_masks, neg_sims, neg_batch_indices = sample_batch['negatives']
    neg_seqs, neg_masks = neg_seqs.to(device), neg_masks.to(device)
    neg_batch_indices = neg_batch_indices.to(device)
    neg_embeddings = model(neg_seqs, neg_masks)
    print(f"Negative embeddings shape: {neg_embeddings.shape}")

    # Check similarity between anchors and positives
    pos_similarities = F.cosine_similarity(embeddings, pos_embeddings, dim=1)
    print(f"Anchor-Positive similarities: {pos_similarities.mean().item():.4f} ± {pos_similarities.std().item():.4f}")

    # Check similarity between anchors and negatives
    batch_size = embeddings.shape[0]
    neg_similarities_all = []

    for i in range(batch_size):
        # Get negatives for this anchor
        neg_mask = neg_batch_indices == i
        if neg_mask.sum() > 0:
            anchor_i = embeddings[i:i+1]  # (1, embedding_dim)
            negatives_i = neg_embeddings[neg_mask]  # (num_negs, embedding_dim)

            # Compute similarities between this anchor and its negatives
            neg_sims_i = F.cosine_similarity(
                anchor_i.expand_as(negatives_i), negatives_i, dim=1
            )
            neg_similarities_all.extend(neg_sims_i.cpu().tolist())

    if len(neg_similarities_all) > 0:
        neg_similarities = torch.tensor(neg_similarities_all)
        print(f"Anchor-Negative similarities: {neg_similarities.mean().item():.4f} ± {neg_similarities.std().item():.4f}")

        # Show the difference (should be positive if model is learning well)
        print(f"Positive vs Negative similarity difference: {pos_similarities.mean().item() - neg_similarities.mean().item():.4f}")
    else:
        print("No negative samples found in this batch")
